# Final Project COMP 603

In [118]:
# Import required modules.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Create Pandas Dataframe using the CSV

In [119]:
columns = ['CustomerID','Year','Week','Volume'] # Define Columns Name Here.

In [120]:
parcel_df = pd.read_csv('COVID_Parcel_Business.csv',names=columns,header=0)
parcel_df.head(10)

,CustomerID,Year,Week,Volume
0,718117,2021,1,8
1,718117,2020,7,257
2,718117,2019,39,141
3,718117,2018,1,14
4,718117,2021,2,192
5,718117,2018,2,18
6,718117,2020,8,232
7,718117,2020,10,229
8,718117,2020,2,251
9,718117,2019,5,93


Lets study the Dataframe to set more info for the Dataset we are using.

Use describe and info to see the Data Type , If there is any missing values.

In [121]:
parcel_df.describe(include="all")

,CustomerID,Year,Week,Volume
count,190719.000000,190719.000000,190719.000000,190719.000000
mean,541440.234963,2019.395220,25.676377,1174.911519
std,264861.365663,1.068845,14.722539,11586.730666
min,100771.000000,2018.000000,1.000000,1.000000
25%,308930.000000,2018.000000,13.000000,25.000000
50%,529707.000000,2019.000000,25.000000,58.000000
75%,778800.000000,2020.000000,38.000000,162.000000
max,999362.000000,2021.000000,53.000000,640590.000000


In [122]:
parcel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190719 entries, 0 to 190718
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   CustomerID  190719 non-null  int64
 1   Year        190719 non-null  int64
 2   Week        190719 non-null  int64
 3   Volume      190719 non-null  int64
dtypes: int64(4)
memory usage: 5.8 MB


Based on Above info we can say that there is no missing value

Check the number of unique values for all the Columns

In [123]:
for col in parcel_df.columns:
  print(f'Unique Values of {col}: {len(parcel_df[col].unique())}')

Unique Values of CustomerID: 1061
Unique Values of Year: 4
Unique Values of Week: 53
Unique Values of Volume: 8332


## Important Period Table

Lets us 1st add the timeline in the table to define all the important period.
 2019 Important Dates

| **Period**      | **Start Date**     | **End Date**       | **Week Numbers**       |
|------------------|--------------------|--------------------|------------------------|
| Year Start       | January 1, 2019   | January 5, 2019    | Week 1                |
| Year End         | December 29, 2019 | December 31, 2019  | Week 53               |
| Peak Season      | November 4, 2019  | January 20, 2020   | Weeks 45–4 (2019–2020)|

 2020 Important Dates

| **Period**        | **Start Date**     | **End Date**       | **Week Numbers**       |
|--------------------|--------------------|--------------------|------------------------|
| Year Start         | January 1, 2020   | January 4, 2020    | Week 1                |
| Year End           | December 27, 2020 | December 31, 2020  | Week 53               |
| Pre-COVID Period   | January 1, 2020   | Week 15 of 2020 | Weeks 1–15           |
| COVID Observation  | Week 16 of 2020  | Week 53 of 2020 | Weeks 16–53          |
| Peak Season        | November 2, 2020  | January 17, 2021   | Weeks 45–3 (2020–2021)|


## Classify customers based on growth labels


| **Customer Size**        | **Definition**     | **Discount**       
|--------------------|--------------------|--------------------|
| Enterprise       | 2019 parcel volume > 500k pieces  | 22%   |
| Large          | 2019 parcel volume b/w 200k-500k | 17%  | 
| Medium  | 2019 parcel volume b/w 10k-200k   | 10% | 
| Small  | 2019 parcel volume b/w 1k-20k  | 4% | 


Steps to Add Define the Customer Label to each Customers. 
1. Get all the data for 2019.
2. Get Sum of Volume of Each Customer and store in total_volume_per_customer.
3. Add a new column Customer Size in Data frame.
4. Based on total_volume_per_customer get the total volume and assign customer Size based on it.

In [124]:
# Return Filtered DataFrame for mention period based 
def get_parcel_by_period(startYear, startWeek, endYear, endWeek):
    # Filter for rows within the start year and week
    start_period = (parcel_df['Year'] > startYear) | ((parcel_df['Year'] == startYear) & (parcel_df['Week'] >= startWeek))
    
    # Filter for rows within the end year and week
    end_period = (parcel_df['Year'] < endYear) | ((parcel_df['Year'] == endYear) & (parcel_df['Week'] <= endWeek))
    
    # Apply both filters
    filtered_df = parcel_df[start_period & end_period]
    
    return filtered_df


In [125]:
# Get all the parcel for 2019.
parcel_2019 = get_parcel_by_period(2019,1,2019,53)
parcel_2019.head(10)

,CustomerID,Year,Week,Volume
2,718117,2019,39,141
9,718117,2019,5,93
12,718117,2019,49,215
19,718117,2019,47,183
21,718117,2019,4,65
25,718117,2019,33,100
30,718117,2019,53,171
32,718117,2019,40,143
38,718117,2019,38,163
46,718117,2019,52,87


In [126]:
# Calculate the total parcel volume for each CustomerID in the given DataFrame.
def calculate_total_volume_per_customer(df):
    
    total_volume_per_customer = df.groupby('CustomerID')['Volume'].sum()
    
    return total_volume_per_customer

In [127]:
total_volume_per_customer = calculate_total_volume_per_customer(parcel_2019)
total_volume_per_customer

CustomerID
100771     1816
102180     5222
102643     2670
102750     1013
103308    90140
          ...  
994361     2608
994762     1073
995173     1632
999141     3364
999362     3561
Name: Volume, Length: 1061, dtype: int64

In [128]:
# This function is to to add Customer Size and Discount for that customers for mentioned year.
def assignCustomerSizeForYear(year):
    column_customer_size_year = f'Customer Size {year}'
    column_discount_year = f'Discount % {year}'

    # Initialize columns in the DataFrame
    parcel_df[column_customer_size_year] = ''  # Add new column for Customer Size
    parcel_df[column_discount_year] = 0  # Add new column for Discount %

    # Get the parcel data for that year
    parcel_year = get_parcel_by_period(year, 1, year, 53)

    # Calculate the total volume per customer for the year
    total_volume_per_customer_year = calculate_total_volume_per_customer(parcel_year)

    # Iterate over the DataFrame rows
    for index, row in parcel_df.iterrows():
        customer_id = row['CustomerID']
        total_volume = total_volume_per_customer_year.get(customer_id, None)  # Get total volume or None if not found                     s

        # Assign customer size and discount based on total volume
        if total_volume is None:
            parcel_df.at[index, column_customer_size_year] = 'Miscellaneous'  # Leave empty if no volume
            parcel_df.at[index, column_discount_year] = 0  # Set discount to 0
        elif total_volume > 500000:
            parcel_df.at[index, column_customer_size_year] = 'Enterprise'
            parcel_df.at[index, column_discount_year] = 22
        elif 200000 < total_volume <= 500000:
            parcel_df.at[index, column_customer_size_year] = 'Large'
            parcel_df.at[index, column_discount_year] = 17
        elif 10000 < total_volume <= 200000:
            parcel_df.at[index, column_customer_size_year] = 'Medium'
            parcel_df.at[index, column_discount_year] = 10
        else:
            parcel_df.at[index, column_customer_size_year] = 'Small'
            parcel_df.at[index, column_discount_year] = 4

For Each year add Customers Size.

In [129]:
years = parcel_df.Year.unique()
for year in years:
    assignCustomerSizeForYear(year)

parcel_df.head(10)

,CustomerID,Year,Week,Volume,Customer Size 2021,Discount % 2021,Customer Size 2020,Discount % 2020,Customer Size 2019,Discount % 2019,Customer Size 2018,Discount % 2018
0,718117,2021,1,8,Small,4,Medium,10,Small,4,Small,4
1,718117,2020,7,257,Small,4,Medium,10,Small,4,Small,4
2,718117,2019,39,141,Small,4,Medium,10,Small,4,Small,4
3,718117,2018,1,14,Small,4,Medium,10,Small,4,Small,4
4,718117,2021,2,192,Small,4,Medium,10,Small,4,Small,4
5,718117,2018,2,18,Small,4,Medium,10,Small,4,Small,4
6,718117,2020,8,232,Small,4,Medium,10,Small,4,Small,4
7,718117,2020,10,229,Small,4,Medium,10,Small,4,Small,4
8,718117,2020,2,251,Small,4,Medium,10,Small,4,Small,4
9,718117,2019,5,93,Small,4,Medium,10,Small,4,Small,4


In [135]:
for year in years:
    # Dynamic column name for the customer size of the given year
    column_customer_size_year = f'Customer Size {year}'
    
    # Group by CustomerID and take the first occurrence of 'Customer Size' for each customer
    unique_customers = parcel_df.groupby('CustomerID').first().reset_index()
    
    # Get the count of each Customer Size for the year
    customer_size_counts = unique_customers[column_customer_size_year].value_counts()

    # Print the counts for the year
    print(f"Customer Size counts for year {year}:\n{customer_size_counts}\n")


Customer Size counts for year 2021:
Customer Size 2021
Small            809
Medium           184
Miscellaneous     47
Enterprise        11
Large             10
Name: count, dtype: int64

Customer Size counts for year 2020:
Customer Size 2020
Small            776
Medium           234
Large             22
Miscellaneous     15
Enterprise        14
Name: count, dtype: int64

Customer Size counts for year 2019:
Customer Size 2019
Small         860
Medium        174
Large          15
Enterprise     12
Name: count, dtype: int64

Customer Size counts for year 2018:
Customer Size 2018
Small            843
Medium           152
Miscellaneous     36
Large             19
Enterprise        11
Name: count, dtype: int64



### Define Customer Growth for Each Customer

In [131]:
def add_growth_label(previous_period_df, current_period_df):
    # Group by CustomerID and calculate total volumes for each period
    previous_volume_per_customer = calculate_total_volume_per_customer(previous_period_df)
    current_volumes_per_customer = calculate_total_volume_per_customer(current_period_df)
    
    # Merge previous and current volumes into a single DataFrame
    growth_data = pd.DataFrame({
        'Previous_Volume': previous_volume_per_customer,
        'Current_Volume': current_volumes_per_customer
    }).fillna(0)  # Fill NaN with 0 for customers not present in one period

    # Calculate ISGR
    growth_data['ISGR (%)'] = (
        (growth_data['Current_Volume'] - growth_data['Previous_Volume']) /
        growth_data['Previous_Volume']
    ) * 100
    growth_data['ISGR (%)'] = growth_data['ISGR (%)'].replace([float('inf'), -float('inf')], None)  # Handle division by zero

    # Determine Growth_Label
    def growth_label(row):
        if row['Previous_Volume'] == 0 and row['Current_Volume'] > 0:
            return 'New Customer'
        elif row['ISGR (%)'] > 0:
            return 'High Growth Customer'
        elif row['ISGR (%)'] == 0:
            return 'Stable Customer'
        elif row['ISGR (%)'] < 0:
            return 'Declining Customer'
        elif row['Previous_Volume'] > 0 and row['Current_Volume'] == 0:
            return 'Lost Customer'
        return 'Unclassified'

    growth_data['Growth_Label'] = growth_data.apply(growth_label, axis=1)
    
    # Add the Growth_Label and ISGR back to the original parcel_df
    parcel_df['Growth_Label'] = parcel_df['CustomerID'].map(growth_data['Growth_Label']).fillna('Unclassified')
    parcel_df['ISGR (%)'] = parcel_df['CustomerID'].map(growth_data['ISGR (%)'])
    
    return parcel_df


In [132]:
previous_period_df = get_parcel_by_period(2019,1,2019,15)
current_period_df = get_parcel_by_period(2020,1,2020,15)
add_growth_label(previous_period_df,current_period_df)
parcel_df.head(10)

,CustomerID,Year,Week,Volume,Customer Size 2021,Discount % 2021,Customer Size 2020,Discount % 2020,Customer Size 2019,Discount % 2019,Customer Size 2018,Discount % 2018,Growth_Label,ISGR (%)
0,718117,2021,1,8,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
1,718117,2020,7,257,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
2,718117,2019,39,141,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
3,718117,2018,1,14,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
4,718117,2021,2,192,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
5,718117,2018,2,18,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
6,718117,2020,8,232,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
7,718117,2020,10,229,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
8,718117,2020,2,251,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496
9,718117,2019,5,93,Small,4,Medium,10,Small,4,Small,4,High Growth Customer,315.323496


In [133]:
parcel_df['Growth_Label'].unique()

array(['High Growth Customer', 'Declining Customer', 'New Customer',
       'Unclassified'], dtype=object)

Now we have Added the Growth label and Customer Size function we can start with plotting the graph.